### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re

#import pymysql
import getpass
#from sshtunnel import SSHTunnelForwarder

from IPython.display import clear_output

import csv
import xlrd
import xlwt

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import unicodedata
import numpy as np
from collections import OrderedDict, Counter
import nltk
from nltk.collocations import *
from nltk.util import ngrams
import string

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
#path = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"

In [3]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows)
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
if os.name == 'nt':
    encoding_type = 'utf-8'
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
else:
    encoding_type = 'ISO-8859-1'
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'

#### Abre arquivo XLS

In [4]:
workbook = xlrd.open_workbook(path+'PESSOAS-do-Accessus.xls', on_demand = True)

#worksheet = workbook.sheet_by_index(0)
worksheet = workbook.sheet_by_name('Rel_DescritorEleito')

WARNING *** file size (1367936) not 512 + multiple of sector size (512)


Captura nomes de entidades pessoas retiradas da base do Accessus/CPDOC

In [33]:
exceptions_list = ['a.', 'i.', 'j.', 'de', 'do']

In [34]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [35]:
names_list = []

for line in range(worksheet.nrows):
    number_name = str(worksheet.cell(line, 0).value)
    check_number = re.search('^\d',number_name)
    if check_number is not None:
        if line == worksheet.nrows-1: break
        fullname = str(worksheet.cell(line, 1).value)
        fullname = str.lower(fullname)
        fullname_split = fullname.split()
        for name in fullname_split: 
            if name not in exceptions_list and name not in names_list:
                alt_name = strip_accents(name)
                names_list.append(name)
                if alt_name != name: names_list.append(alt_name) 

In [8]:
names_list[:10]

['castro',
 'lima',
 'saldanha',
 'júnior',
 'junior',
 'azevedo',
 'faria',
 'peixoto',
 'pinto',
 'ferraz']

In [9]:
#Usar este mesmo modelo para formar a variável de extração de entidade
texts = ''
#list_raw_texts = []
for filename in onlyfiles:
    page_txt = codecs.open(os.path.join(corpuspath,filename), encoding='utf-8').read()
    texts += '\n'+ page_txt.lower()
    #list_raw_texts.append(page_txt)
#len(list_raw_texts)

NameError: name 'onlyfiles' is not defined

In [30]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
alimenta a a tabela person_doc, da nossa base no mysql.
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
texts = ''
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    #cur.execute("DROP TABLE IF EXISTS person_doc")
    #cur.execute('''CREATE TABLE IF NOT EXISTS person_doc 
    #           (person_id INT(11), doc_id VARCHAR(31), person_count BIGINT(21)
    #           DEFAULT NULL)
    #           ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text =  row[1]
        texts += '\r\n'+ text

        #print(name, 'text_id', text_id)
        #print(doc_id, topic_id, topic_score)   
        #cur.execute("ALTER TABLE person_doc ORDER BY doc_id ASC, person_count DESC")
        
        if count == 10: break #amostra de apenas 3 linhas

········
········


In [28]:
texts

'\r\n \n\njr, /h \' "& m$ hb\'h\'qo\n& la\'?\n\n, é %%3 - _\n(aonsulado do brasil _/ , ova/caio\n4 & ,\n* brazilian consulate wo \'\naas hawthorn st. 16 % wja mm\nmu. .. " "\nmm .\n\nnew bedford, massachuseti\'s. u. s, a, 02740\n\n \n\nmarcio muller bueno\n" , /\n11:11:3sz mao jz". www não"3337552 %%, " aca-m\n)\nwha %" wom- aba/và %%\n\nxb" 7/35 andado.\n\n_, íáaw mí. m luz/á && %% %%, aj alma, ;\n(\n\n"me/&& % \'wl-(aajj-mi- # "a" aaa/fo &\n"não. aiwa %%%&/a %%%&/www a* m" á?!" á\n1/1"me %"aí\'ej\'wfo m/k%/ám%â tai/á\nw. : úwá 2-- /i < %%\n"melwa/ volíco ay? mul/1 "( o , (-\n\n\' % ó<\n/l ano)/4% wklt-w wa: (lw/ám/[a w\n&& mwráww/láw wa",\n\n/-\n\númháo , (; vzííóo quo/\'. ,\n"ac-"44. \' pá , (".\nv £ f (1. é : "l 01; mw w wm %- &\n/\n\nwl] [iv-156%; , (, \'\n"ab/q "já-m m\x80) kífa-nmc aaa-x mw, ala\nwww,\n%"wáwí, 9 m: ( c/mn\n\n%sjjéãm\n\n"ém-k w [\', /á" "**-"í; pé ifá?!" w %\'0\nm amo. .. /lilia"? jmá%. %i/w %\n&& po "vpmtãvak_ jun moo-"n & %% "4/4, "\n\np (aõ piatã h , azo"wat a

In [41]:
#query = 'azeredo'
for name in names_list:
    query = name
    tknzr = nltk.tokenize.WordPunctTokenizer()
    tokens = tknzr.tokenize(texts) # texto com stopwords
    #tokens = tknzr.tokenize(cleaned_texts) # texto sem stopwords
    nltk_text = nltk.Text(tokens)
    nltk_text.concordance(query.lower(), width=50, lines=10)

Displaying 4 of 4 matches:
semb para embaixador azeredo da silveira e sra bu
iq l r :: embbixador azeredo silveira <= av vieir
 urgent , embaixador azeredo silveira vieira sout
mmm de =' ehbaixador azeredo da silveira <= a - v


In [38]:
type(nltk_text)

list

In [15]:
nltk_text.similar(query)

In [19]:
nltk_text.collocations(num=20, window_size=2)

marcio muller; new bedford; brazilian consulate; muller bueno


Collocations are expressions of multiple words which commonly co-occur.
Below we are using Pointwise Mutual Information.
http://en.wikipedia.org/wiki/Pointwise_mutual_information

In [20]:
max_items = 50
freq_min_b = 2
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = nltk.collocations.BigramCollocationFinder.from_words(nltk_text)
finder2.apply_word_filter(lambda w: w in string.punctuation)
finder2.apply_freq_filter(freq_min_b)
for a, b in finder2.nbest(bigram_measures.pmi, max_items):
    print (a, b)

marcio muller
new bedford
brazilian consulate
muller bueno


In [21]:
max_items = 50
freq_min_t = 2
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder3 = nltk.collocations.TrigramCollocationFinder.from_words(nltk_text)
finder3.apply_word_filter(lambda w: w in string.punctuation)
finder3.apply_ngram_filter(lambda w1, w2, w3:  w1 in ['da', 'de', 'das'])
finder3.apply_freq_filter(freq_min_t)
for a, b, c in finder3.nbest(trigram_measures.pmi, max_items):
    print (a, b, c)

marcio muller bueno


In [22]:
for a, b in (finder2.above_score(bigram_measures.raw_freq,1.0 / len(list(nltk.bigrams(tokens))))):
        print (a, b)

brazilian consulate
marcio muller
muller bueno
new bedford


In [23]:
for a, b, c in finder3.above_score(trigram_measures.raw_freq,1.0 / len(list(nltk.trigrams(tokens)))):
        print (a, b, c)

marcio muller bueno


### Inserção na base de dados

In [24]:
n = 4
xgrams = ngrams(tokens, n)
xgrams_counter = Counter(xgrams)
series_xgrams_counter = pd.Series(data=xgrams_counter, index=xgrams_counter.keys())
df_ngrams = pd.DataFrame(series_xgrams_counter, columns = ['Frequência'])
df_ngrams = df_ngrams.sort_values(by='Frequência', ascending=False)
df_ngrams.index.name = 'Tokens'
df_ngrams[0:40]

,Frequência
Tokens,
"(&&, %%, %%,, aj)",1
"(vão, :/&?, c, "")",1
"(&, ,, *, brazilian)",1
"(,, "", p, ()",1
"(á, ?!"", á, 1)",1
"(xb, "", 7, /)",1
"(-(, aajj, -, mi)",1
"(,, a, ,, 02740)",1
"(/, j, ,, ,)",1


# RASCUNHOS

In [146]:
cfd = nltk.ConditionalFreqDist(
    (target, fileid[:4])
    for fileid in inaugural.fileids()
    for w in inaugural.words(fileid)
    for target in ['america', 'citizen']
    if w.lower().startswith(target)) [1]
cfd.plot()

NameError: name 'inaugural' is not defined

##### ENTIDADE: PESSOA

Alimenta a tabela person_doc, da nossa base no mysql, fazendo interseção de dados da tabela persons com a tabela docs.

In [12]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
alimenta a a tabela person_doc, da nossa base no mysql.
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS person_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS person_doc 
               (person_id INT(11), doc_id VARCHAR(31), person_count BIGINT(21)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text_id = row[0]
        text =  row[1]
        #text = text.split()
        doc_id = row[0]
        for name in names_list_test:
            person_id = 1
            person_count = text.count(name)
            #query = "INSERT INTO person_doc VALUES (%s, %s, %s)"
            #cur.execute(query, (person_id, doc_id, person_count))
        print(name, 'text_id', text_id)
        #print(doc_id, topic_id, topic_score)   
        #cur.execute("ALTER TABLE person_doc ORDER BY doc_id ASC, person_count DESC")
        
        if count == 2: break #amostra de apenas 10 linhas

········
········
azeredo text_id rb_1974.04.17_doc_I-66
azeredo text_id onu_1974.03.12_doc_II-49
azeredo text_id onu_1974.03.12_doc_V-2
azeredo text_id be_1974.04.16_doc_VI-18
azeredo text_id be_1977.06.01_doc_II-21
azeredo text_id ag_1974.03.13_doc_III-7
azeredo text_id ag_1973.11.20_doc_VI-75
azeredo text_id d_1974.04.23_doc_XXXII-8
azeredo text_id d_1974.03.26_doc_XXXII-32
azeredo text_id d_1974.03.26_doc_XXII-9


##### ENTIDADE: LUGAR (PAÍS)

Alimenta a a tabela country_doc, da nossa base no mysql, fazendo interseção de dados da tabela countries com a tabela docs.